In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [2]:
%load_ext Cython

In [3]:
%%cython
cimport cython
cimport numpy as np
import  numpy as np
from libc.math cimport sqrt


@cython.cdivision(True)
@cython.boundscheck(False)
@cython.wraparound(False)
cdef int _rotation(double [:,::1] input_data, double [:,::1] output_data,
                  bint interpolation = 0, (Py_ssize_t, Py_ssize_t) interval):
    cdef Py_ssize_t r_len, s_len
    r_len = input_data.shape[1]
    s_len = input_data.shape[0]
    cdef Py_ssize_t r_len_half = r_len // 2
        
    cdef Py_ssize_t yy, xx ,rr
    cdef double radius 
    cdef double summed_line_up, summed_line_down
    cdef double interpolated_value_up, interpolated_value_down
    cdef double value_diff_up, value_diff_down
    
    cdef double* value_1_down
    cdef double* value_1_up
    cdef double* value_2_down
    cdef double* value_2_up
    # no interpolation
    # working on interpolation 
    
    cdef Py_ssize_t offset, loop_start
    if  r_len%2 == 0:
        offset = 1
        loop_start = 0
    else:
        offset = 0
        loop_start = 1
        
        yy = 0
        summed_line_up = 0
        # just the middle line 
        if not interpolation:
            for zz in range(s_len):
                for xx in range(interval[0] - yy, interval[1] , -1):
                    radius = sqrt((yy + 0.25)**2 + (xx - 0.5)**2)
                    rr = int(radius)
                    summed_line_up += (input_data[zz,r_len_half  + rr] 
                                        * (yy + 0.25)/radius)
                output_data[r_len_half - yy ,zz] = summed_line_up
        else:
            for zz in range(s_len):
                for xx in range(r_len_half - yy, 0 , -1):
                    radius = sqrt((yy + 0.25)**2 + (xx - 0.5)**2)
                    rr = int(radius)
                    summed_line_up += (input_data[zz,r_len_half  + rr] 
                                       * (yy + 0.25)/radius)
                    output_data[r_len_half - yy ,zz] = summed_line_up
                    value_1_up = &input_data[zz,r_len_half + rr]
                    if r_len_half - 1 == rr:
                        interpolated_value_up = value_1_up[0]
                    else: 
                        value_2_up = &input_data[zz,r_len_half + rr + 1]
                        value_diff_up = value_2_up[0] - value_1_up[0]
                        interpolated_value_up = (value_1_up[0] + 
                                                     value_diff_up
                                                     * (radius - rr))
                    summed_line_up += (interpolated_value_up
                                       * (yy + 0.25)/radius)
                output_data[r_len_half - yy ,zz] = summed_line_up
            
    if interpolation:
        for zz in range(s_len):
            for yy in range(loop_start, r_len_half):
                summed_line_up = 0
                summed_line_down = 0
                if r_len_half - yy <= interval[0]:
                    x_start  = r_len_half - yy
                else:
                    x_start = interval[0]
                for xx in range(x_start, interval[1] , -1):
                    radius = sqrt((yy + 0.5)**2 + (xx - 0.5)**2)
                    rr = int(radius)
                    value_1_up = &input_data[zz,r_len_half + rr]
                    value_1_down = &input_data[zz, r_len_half - offset - rr]
                    if r_len_half - 1 == rr:
                        interpolated_value_up = value_1_up[0]
                        interpolated_value_down = value_1_down[0]
                    else: 
                        value_2_up = &input_data[zz,r_len_half + rr +1]
                        value_2_down = &input_data[zz,r_len_half  - offset - rr - 1]

                        value_diff_up = value_2_up[0] - value_1_up[0]
                        value_diff_down = value_2_down[0] - value_1_down[0]

                        interpolated_value_up = (value_1_up[0] + 
                                                 value_diff_up
                                                 * (radius - rr))
                        interpolated_value_down = (value_1_down[0] +
                                                   value_diff_down
                                                   * (radius - rr))

                    summed_line_up += (interpolated_value_up 
                                       * (yy + 0.5)/radius)
                    summed_line_down += (interpolated_value_down 
                                         * (yy + 0.5)/radius)
                # multiply with 2 to include the other side (It's identical)
                output_data[r_len_half - offset -yy ,zz] = 2 * summed_line_up
                output_data[r_len_half +  yy, zz]  = 2 * summed_line_down
    else:
        for zz in range(s_len):
            for yy in range(loop_start, r_len_half):
                summed_line_up = 0
                summed_line_down = 0
                if r_len_half - yy <= interval[0]:
                    x_start  = r_len_half - yy
                else:
                    x_start = interval[0]
                for xx in range(x_start, interval[1], -1):
                    radius = sqrt((yy + 0.5)**2 + (xx - 0.5)**2)
                    rr = int(radius)

                    value_1_up = &input_data[zz,r_len_half + rr]
                    value_1_down = &input_data[zz, r_len_half - offset - rr]

                    interpolated_value_up = value_1_up[0]
                    interpolated_value_down = value_1_down[0]

                    summed_line_up += (interpolated_value_up 
                                       * (yy + 0.5)/radius)
                    summed_line_down += (interpolated_value_down 
                                         * (yy + 0.5)/radius)
                # multiply with 2 to include the other side (It's identical)
                output_data[r_len_half - offset -yy ,zz] = 2 * summed_line_up
                output_data[r_len_half +  yy, zz]  = 2 * summed_line_down

    return loop_start

def rotation(input_data, interpolation = False):
    try:
        input_data.flags
    except AttributeError:
        print('input_data should be a numpy array!')
        raise
    if input_data.flags['F_CONTIGUOUS']:
        print('input_data should be stored in a raw major, C contigous.')
    elif not input_data.flags['C_CONTIGUOUS']:
        print('input_data as to be a contigous array.')
    
    if input_data.dtype != np.float64:
        print('input should be an ndarray of type np.float64.')
        print('Converting to float64 and continuing...')
        input_data = input_data.astype(np.float64)

    output = np.zeros(input_data.size, dtype=np.float64, order='C').reshape(
                    input_data.shape[1], input_data.shape[0])
    
    cdef double [:,::1] input_data_view = input_data
    cdef double [:,::1] output_view = output
    # returned_status just for debugging, flow control. To be removed!
    returned_status = _rotation(input_data_view, output_view, interpolation)
    print(returned_status)
    return output

def rotation_timeresolved( steps, intervals, interpolation = False):
    steps.size == intervals.size
    cdef double [:,::1] step_view
    for step in steps:
        try:
            step.flags
        except AttributeError:
            print('input_data should be a numpy array!')
            raise
        if input_data.flags['F_CONTIGUOUS']:
            print('input_data should be stored in a raw major, C contigous.')
        elif not input_data.flags['C_CONTIGUOUS']:
            print('input_data as to be a contigous array.')

        if input_data.dtype != np.float64:
            print('input should be an ndarray of type np.float64.')
            print('Converting to float64 and continuing...')
            input_data = input_data.astype(np.float64)
            step_view  = step
        
    
    return 0


Error compiling Cython file:
------------------------------------------------------------
...
    # returned_status just for debugging, flow control. To be removed!
    returned_status = _rotation(input_data_view, output_view, interpolation)
    print(returned_status)
    return output

def rotation_timeresolved([:,::1] [:] steps, intervals, interpolation):
                         ^
------------------------------------------------------------

/home/pawel/.cache/ipython/cython/_cython_magic_24a983debf49cbde59772a9740456542.pyx:163:26: Expected an identifier, found '['

Error compiling Cython file:
------------------------------------------------------------
...
    # returned_status just for debugging, flow control. To be removed!
    returned_status = _rotation(input_data_view, output_view, interpolation)
    print(returned_status)
    return output

def rotation_timeresolved([:,::1] [:] steps, intervals, interpolation):
                           ^
---------------------------------

TypeError: object of type 'NoneType' has no len()